# Projeto de Extração de Dados de Engenharia de Dados da ADA Tech

# 1. Bibliotecas

As bibliotecas utilizadas para criação do Webhook são:

- Requests: Pacote de requisição GET.
- OS: Gestão de pastas do Windows.
- Flask: Pacote Python para criação de aplicação WEB.
- Dotenv: Pacote Python para resguardar as variáveis de ambiente.

In [1]:
import requests
import os
from flask import Flask,request
from dotenv import load_dotenv

# 2. Criação do webhook e Chamada

### 2.1 Webhook

In [2]:
load_dotenv()

app = Flask(__name__)

NEWS_API_KEY = os.getenv('news_api')

@app.route('/webhook', methods=['POST'])
def webhook():
    if request.method == 'POST':
        data = request.json
        if 'query' in data:
            query = data['query']
            one_hour_ago = data['from']
            current_time = data['to']
            news_data = get_news(query,one_hour_ago,current_time)
            return news_data
        else:
            return "Query não encontrada no payload."

def get_news(query,one_hour_ago,current_time):
    url = f'https://newsapi.org/v2/everything?q={query}&from={one_hour_ago}&to={current_time}&apiKey={NEWS_API_KEY}'
    response = requests.get(url)
    
    if response.status_code == 200:
        news_data = response.json()
        return news_data
    else:
        return "Erro ao buscar notícias."

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=8000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.15.18:8000
Press CTRL+C to quit
192.168.15.18 - - [01/Apr/2024 20:00:01] "POST /webhook HTTP/1.1" 200 -
192.168.15.18 - - [01/Apr/2024 21:00:01] "POST /webhook HTTP/1.1" 200 -
192.168.15.18 - - [01/Apr/2024 21:29:58] "POST /webhook HTTP/1.1" 200 -
192.168.15.18 - - [01/Apr/2024 21:31:20] "POST /webhook HTTP/1.1" 200 -
192.168.15.18 - - [01/Apr/2024 21:32:32] "POST /webhook HTTP/1.1" 200 -
192.168.15.18 - - [01/Apr/2024 21:32:38] "POST /webhook HTTP/1.1" 200 -
192.168.15.18 - - [01/Apr/2024 21:35:28] "POST /webhook HTTP/1.1" 200 -


### 2.2 Chamada de API

In [ ]:
def requisicao(query,one_hour_ago, current_time):   
    # URL da API webhook
    url = "http://192.168.15.18:8000/webhook"
    
    # Dados a serem enviados na requisição
    data = {
        "query": query,
        "from":one_hour_ago,
        "to":current_time
    }

    # Enviando a requisição POST para a API
    response = requests.post(url, json=data)


    # Verificando o status da resposta
    if response.status_code == 200:

        # Geração do arquivo JSON
        webhook_data = response.json()

        if webhook_data["totalResults"] > 0:

            dia_anterior = (datetime.now() - timedelta(days=1) - timedelta(hours=1)).strftime('%Y-%m-%d')

            hm = (datetime.now() - timedelta(hours=1)).strftime('%H-%M')

            folder_path = f'noticias/{dia_anterior}'
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            with open(f'{folder_path}/{hm}-news.json', 'w') as f:
                json.dump(webhook_data, f)
            print("Novas notícias foram encontradas e salvas:", webhook_data)

        else:
            print("Não há noticias novas com essa Query")
        
    else:
        # Caso contrário, houve um erro na requisição
        print("Erro ao enviar webhook.")
        if not os.path.exists("./log_erro.txt"):
            with open(f'log_erro.txt', 'w') as f:
                f.write(f"{hm},{response.text}")
        with open(f'news.json', 'a') as f:
            f.write(f"{hm},{response.text}")

